In [32]:
# Trying to scrap data from Steam RPG games
# Questions created:
# https://stackoverflow.com/questions/49035188/finding-number-of-pages-using-python-beautifulsoup
# https://stackoverflow.com/questions/49019168/using-beautiful-soup-in-a-class-that-contains-spaces

import requests, re
from bs4 import BeautifulSoup

In [33]:
r = requests.get("http://store.steampowered.com/tags/en-us/RPG/")
c = r.content
soup = BeautifulSoup(c, 'html.parser')
total_items = int(soup.find("span",{"id":"NewReleases_total"}).text)
items_per_page = int(soup.find('span', id='NewReleases_end').text)
page_quantity = round(total_items/items_per_page) # Total pages containing 20 games each

In [34]:
games_list = []
base_url_begin = 'http://store.steampowered.com/tags/en-us/RPG/#p='
base_url_finish = '&tab=NewReleases'
for page in range(0, page_quantity):
    r = requests.get(base_url_begin + str(page) + base_url_finish)
    c = r.content
    soup = BeautifulSoup(c,'html.parser')
    all_games = soup.find('div', {'id':'NewReleasesRows'}).find_all('a', {'class':'tab_item'}) # Main games from page
    for game in all_games:
        d = {}
        d['Name'] = game.find('div', {'class':'tab_item_name'}).text
        d['Price'] = game.find('div', {'class':'discount_final_price'}).text
        try:
            d['Discount'] = game.find('div', {'class':'discount_pct'}).text
        except:
            d['Discount'] = str(0)
        tags_list = game.find('div', {'class':'tab_item_top_tags'}).text.replace(',', '').split()
        d['Tags'] = tags_list
        platforms_list = []
        for platform in game.find_all('span', {'class':'platform_img'}):
            platforms_list.append(platform['class'][1])
        d['Platforms'] = platforms_list
        games_list.append(d)
        

In [35]:
import pandas
df=pandas.DataFrame(games_list)

In [36]:
df.head() # Looks fine

,Discount,Name,Platforms,Price,Tags
0,-32%,PLATI NALOG: Favorite Russian Game,[win],"R$ 1,55","[Strategy, RPG, Indie, Casual]"
1,0,Blocked and Loaded,[win],"R$ 2,29","[Massively, Multiplayer, Indie, Action, Advent..."
2,-15%,Where the Water Tastes Like Wine,"[win, mac, linux]","R$ 31,44","[Indie, Adventure, RPG, Great, Soundtrack]"
3,-10%,Elium - Prison Escape,[win],"R$ 25,10","[Indie, Violent, RPG, Action]"
4,0,众生,[win],"R$ 28,99","[Early, Access, Strategy, RPG, Indie]"


In [37]:
df = df[['Name','Price','Discount','Tags','Platforms']] # Rearranging

In [38]:
df.head()

,Name,Price,Discount,Tags,Platforms
0,PLATI NALOG: Favorite Russian Game,"R$ 1,55",-32%,"[Strategy, RPG, Indie, Casual]",[win]
1,Blocked and Loaded,"R$ 2,29",0,"[Massively, Multiplayer, Indie, Action, Advent...",[win]
2,Where the Water Tastes Like Wine,"R$ 31,44",-15%,"[Indie, Adventure, RPG, Great, Soundtrack]","[win, mac, linux]"
3,Elium - Prison Escape,"R$ 25,10",-10%,"[Indie, Violent, RPG, Action]",[win]
4,众生,"R$ 28,99",0,"[Early, Access, Strategy, RPG, Indie]",[win]


In [39]:
df[df['Name'] == 'Elium - Prison Escape'] # There is something wrong, 
# somehow games are being repeated, I'll discover the cause and fix it thereafter

,Name,Price,Discount,Tags,Platforms
3,Elium - Prison Escape,"R$ 25,10",-10%,"[Indie, Violent, RPG, Action]",[win]
23,Elium - Prison Escape,"R$ 25,10",-10%,"[Indie, Violent, RPG, Action]",[win]
43,Elium - Prison Escape,"R$ 25,10",-10%,"[Indie, Violent, RPG, Action]",[win]
63,Elium - Prison Escape,"R$ 25,10",-10%,"[Indie, Violent, RPG, Action]",[win]
83,Elium - Prison Escape,"R$ 25,10",-10%,"[Indie, Violent, RPG, Action]",[win]
103,Elium - Prison Escape,"R$ 25,10",-10%,"[Indie, Violent, RPG, Action]",[win]
123,Elium - Prison Escape,"R$ 25,10",-10%,"[Indie, Violent, RPG, Action]",[win]
143,Elium - Prison Escape,"R$ 25,10",-10%,"[Indie, Violent, RPG, Action]",[win]
163,Elium - Prison Escape,"R$ 25,10",-10%,"[Indie, Violent, RPG, Action]",[win]
183,Elium - Prison Escape,"R$ 25,10",-10%,"[Indie, Violent, RPG, Action]",[win]


In [40]:
df.to_csv('Output_trying_by_myself.csv') # Creates a csv file